In [ ]:
# Note that this script used on Google Colab. For other environments, some modifications might be required.

In [ ]:
!nvidia-smi

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')



In [ ]:
# prompt: I want to mount it to my google drive

runtime = os.getcwd()
HOME = os.path.join(runtime, "drive", "MyDrive", "BottleCap")
print(HOME)



In [ ]:

!pip install ultralytics -q

from IPython import display

display.clear_output()

import ultralytics
ultralytics.checks()

from ultralytics import YOLO
from IPython.display import display, Image

In [ ]:
!mkdir -p {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="YOUR API KEY")
project = rf.workspace("anil-bayram").project("bottle_cap-wvfkb")
version = project.version(3)
dataset = version.download("yolov11")




In [ ]:
# # prompt: I want to delete runs folder. Note that it is not empty

# import shutil
# import os


# delete_dir = '/content/drive/MyDrive/BottleCap/models/yolo11m.pt'
# if os.path.exists(delete_dir):
#     try:
#         shutil.rmtree(delete_dir)
#         print(f"Successfully deleted '{delete_dir}' folder and its contents.")
#     except OSError as e:
#         print(f"Error deleting '{delete_dir}': {e}")
# else:
#     print(f"'{delete_dir}' folder does not exist.")

In [ ]:
# prompt: Create a folder inside a HOME whose name is models
import os

models_dir = os.path.join(HOME, "models_cap2")

# Create the models directory if it doesn't exist
os.makedirs(models_dir, exist_ok=True)

In [ ]:
model_name = 'yolo11s.pt'
model_dir = os.path.join(models_dir, model_name)
os.makedirs(model_dir, exist_ok=True)
%cd {model_dir}
!yolo task=detect mode=train model={model_name} data={dataset.location}/data.yaml epochs=100 imgsz=640 plots=True device=cuda batch=32

In [ ]:
%cd {model_dir}
Image(filename=f'{model_dir}/runs/detect/train/confusion_matrix.png', width=600)

In [ ]:
%cd {model_dir}
Image(filename=f'{model_dir}/runs/detect/train/results.png', width=600)

In [ ]:
%cd {model_dir}
!yolo task=detect mode=predict model={model_dir}/runs/detect/train/weights/best.pt conf=0.6 source={dataset.location}/test/images save=True

In [ ]:
import glob
from IPython.display import Image, display

# Define the base path where the folders are located
base_path = os.path.join(model_dir, 'runs','detect')

# List all directories that start with 'predict' in the base path
subfolders = [os.path.join(base_path, d) for d in os.listdir(base_path)
              if os.path.isdir(os.path.join(base_path, d)) and d.startswith('predict')]

# Find the latest folder by modification time
latest_folder = max(subfolders, key=os.path.getmtime)

image_paths = glob.glob(f'{latest_folder}/*.jpg')[:]

# Display each image
for image_path in image_paths:
    display(Image(filename=image_path, width=600))
    print("\n")

In [ ]:
%cd {model_dir}
!yolo task=detect mode=val model={model_dir}/runs/detect/train/weights/best.pt data={dataset.location}/my_data.yaml